# Brick-image crawling

## [PART 1] 대표 이미지 수집

#### 0. structure-flow

1. 페이지 접속
2. 페이지에서 ul 컨테이너를 찾는다. class="products columns-3"
3. ul 컨테이너 내부에 각 div로 total-product-image-wrap class를 찾는다
페이지당 이미지는 최대 21개이다. if문으로 21인 경우, 다음 페이지로 넘어가도록 설계하자
4. a href를 따라가서 두번째 이미지의 xpath를 찾는다.
5. 이미지를 수집하고 다시 이전 페이지로 돌아간다.
6. cnt로 현재까지 수집한 이미지의 개수를 센 다음에, % 나머지 값을 구해서 다음 페이지로 넘어간다

1->2->(3->4->5->6)과정 반복


#### 1. 웹 크롤링 모듈 불러오기 (webdriver + Selenium + BeautifulSoup)

In [1]:
# selenium settings
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [742 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelea

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from urllib.request import urlopen
from urllib.parse import quote_plus
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
import urllib
import os

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### 2. 대표 레고 이미지 url수집

In [4]:
url = 'https://minifigs.blog/collection/' #기본 경로 설정
filepath = input('원하는 이미지 페이지 번호를 입력하세요 : ')
if not filepath:
  base_url = url
else:
  base_url = url + 'page/' + filepath.strip() + '/'
print("현재 경로 설정 : ", base_url)

원하는 이미지 페이지 번호를 입력하세요 : 
현재 경로 설정 :  https://minifigs.blog/collection/


In [5]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=chrome_options)
driver.get(base_url)

In [6]:
cnt = input('수집하고자 하는 이미지 수량을 입력하세요 : ')
if cnt and cnt != 'pass':
    cnt = int(cnt)
else:
  cnt = 2
print("현재 수집 예정 이미지 수량 : ", cnt)

수집하고자 하는 이미지 수량을 입력하세요 : 100
현재 수집 예정 이미지 수량 :  100


In [7]:
# 드라이버 새로고침, 화면 로드 지연 해결
common_path1 = '//*[@id="primary"]/ul/li'
common_path2 = '/div[1]/a'
#수집하고자 하는 이미지 url 담는 리스트
image_url_list = []

def find_lego(page, idx):
    #collections내 레고 이미지 순차적으로 탐색
    img_container = driver.find_element(by=By.XPATH, value=common_path1 + "[{idx}]".format(idx = str(idx)) + common_path2)

    img_container.click()
    driver.switch_to.frame(0)
    #정보 페이지에서 10도 간격으로 시점을 이동하는 이미지 선택
    for i in range(1, 37):
      second_img_path = '/html/body/div[2]/div[1]/img[{i}]'.format(i=i)
      lego = driver.find_element(by=By.XPATH, value=second_img_path)
      lego = lego.get_attribute('src')
      image_url_list.append(lego) #이미지 경로 수집 완료
    print("현재 페이지 :", page+1, idx, "현재 수집하는 레고 이미지 경로 :", lego)
    

    #페이지 이전으로 이동하기
    driver.back()
    time.sleep(1)

In [8]:
for i in range(0, cnt):
  current_page = i // 21
  if i < 21: #20까지는 동일한 페이지 내에 존재하는 이미지 수집
    driver.refresh()
    time.sleep(1)

    find_lego(current_page, i+1)
  elif i >= 21 and i % 21 == 0:
    #페이지 다음으로 이동하기
    driver.refresh()
    time.sleep(1)

    #ul의 마지막 자식 노드를 탐색(>)
    next_num = '/html/body/div[1]/div/div/div[1]/nav/ul/li[last()]/a'
    try:
      page_next = driver.find_element(by=By.XPATH, value=next_num)
    except:
      break
    page_next.click()
    find_lego(current_page, i % 21 +1)
  else:
    driver.refresh()
    time.sleep(1)

    find_lego(current_page, i % 21 +1)

현재 페이지 : 1 1 현재 수집하는 레고 이미지 경로 : https://minifigs.blog/360/03416/img/01-36.jpg
현재 페이지 : 1 2 현재 수집하는 레고 이미지 경로 : https://minifigs.blog/360/03415/img/01-36.jpg
현재 페이지 : 1 3 현재 수집하는 레고 이미지 경로 : https://minifigs.blog/360/03414/img/01-36.jpg
현재 페이지 : 1 4 현재 수집하는 레고 이미지 경로 : https://minifigs.blog/360/03413/img/01-36.jpg
현재 페이지 : 1 5 현재 수집하는 레고 이미지 경로 : https://minifigs.blog/360/03412/img/01-36.jpg
현재 페이지 : 1 6 현재 수집하는 레고 이미지 경로 : https://minifigs.blog/360/03411/img/01-36.jpg
현재 페이지 : 1 7 현재 수집하는 레고 이미지 경로 : https://minifigs.blog/360/03410/img/01-36.jpg
현재 페이지 : 1 8 현재 수집하는 레고 이미지 경로 : https://minifigs.blog/360/03409/img/01-36.jpg
현재 페이지 : 1 9 현재 수집하는 레고 이미지 경로 : https://minifigs.blog/360/03408/img/01-36.jpg
현재 페이지 : 1 10 현재 수집하는 레고 이미지 경로 : https://minifigs.blog/360/03407/img/01-36.jpg
현재 페이지 : 1 11 현재 수집하는 레고 이미지 경로 : https://minifigs.blog/360/03406/img/01-36.jpg
현재 페이지 : 1 12 현재 수집하는 레고 이미지 경로 : https://minifigs.blog/360/03405/img/01-36.jpg
현재 페이지 : 1 13 현재 수집하는 레고 이미지 경로 : https://minifig

관련 참고 자료 : https://github.com/JeongJaeyoung0/crawling/blob/f75c8d0a92c01c0a1813da4eb8e38555f4e82652/210708_crawler_google_image(colab).ipynb

In [11]:
print('총 수집 완료된 이미지 수 :', len(image_url_list) / 36)

총 수집 완료된 이미지 수 : 100.0


In [12]:
from copy import deepcopy
bck = deepcopy(image_url_list)
bck

['https://minifigs.blog/360/03416/img/01-01.jpg',
 'https://minifigs.blog/360/03416/img/01-02.jpg',
 'https://minifigs.blog/360/03416/img/01-03.jpg',
 'https://minifigs.blog/360/03416/img/01-04.jpg',
 'https://minifigs.blog/360/03416/img/01-05.jpg',
 'https://minifigs.blog/360/03416/img/01-06.jpg',
 'https://minifigs.blog/360/03416/img/01-07.jpg',
 'https://minifigs.blog/360/03416/img/01-08.jpg',
 'https://minifigs.blog/360/03416/img/01-09.jpg',
 'https://minifigs.blog/360/03416/img/01-10.jpg',
 'https://minifigs.blog/360/03416/img/01-11.jpg',
 'https://minifigs.blog/360/03416/img/01-12.jpg',
 'https://minifigs.blog/360/03416/img/01-13.jpg',
 'https://minifigs.blog/360/03416/img/01-14.jpg',
 'https://minifigs.blog/360/03416/img/01-15.jpg',
 'https://minifigs.blog/360/03416/img/01-16.jpg',
 'https://minifigs.blog/360/03416/img/01-17.jpg',
 'https://minifigs.blog/360/03416/img/01-18.jpg',
 'https://minifigs.blog/360/03416/img/01-19.jpg',
 'https://minifigs.blog/360/03416/img/01-20.jpg',


#### 3. 레고 이미지 구글 드라이브에 저장

In [13]:
#레고 이미지 저장 경로
if not os.path.exists("/content/drive/MyDrive/images"):
    os.makedirs("/content/drive/MyDrive/images")

In [14]:
opener = urllib.request.build_opener()
opener.addheaders = [('User-Agent','Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1941.0 Safari/537.36')]
urllib.request.install_opener(opener)

In [ ]:
# 이미지 저장
img_num = 0
count = 0
err_img_url = []
keyword = 'lego'

for n, i in enumerate(image_url_list):
  try:
    if count == 0:
      img_num += 1
    urllib.request.urlretrieve(i, "/content/drive/MyDrive/images/{0}_{1}_{2}.jpg".format(keyword, img_num, count))
    if count == 35:
      count = 0
    else:
      count += 1
    if n!=0 and n%36==0:
        print(n/36, "/", len(image_url_list) / 36, "개 완료")
  except:
    err_img_url.append(i)
    print("error 발생으로 수집안됨 (번호: %d)" % n)   # 주로 503 error, error 발생시 수동으로 이미지 저장 (  img_url[error No.]  )
print('\n저장된 이미지 개수 :', img_num, '\n')
print('에러 이미지 URL:')
for i in err_img_url:
    print(i)

In [17]:
print(img_num)

100
